RAG Piplelines - Data Ingestion to Vector DB Pipleline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path


In [ ]:

def process_all_pdfs(pdf_directory):
    """ Process all the pdf files in a directory"""

    all_document = [ ]
    pdf_dir = Path(pdf_directory)

    # Find all PDF files recursively
    pdf_files = list (pdf_dir.glob('**/*.pdf'))

    print(f"Found {len(pdf_files)} PDF files to process.")
    for pdf_file in pdf_files:
        print(f"\nProcessing {pdf_file.name}")

        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            #Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'

            all_document.extend(documents)
            print(f" Loaded {len(documents)} pages")
        except Exception as e:
            print(f" Error: {e}")

    print(f"\n Total documents loaded:{len(all_document)}")



